<a href="https://colab.research.google.com/github/JustJake451/BAN_403_Group_Project/blob/main/BAN_403_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q amplpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 47.0 MB/s eta 0:00:00


In [ ]:
from amplpy import AMPL, tools

ampl = tools.ampl_notebook(
    modules=["coin"],
    license_uuid="default")

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


$x_{wo}$ = 1 if we play team $o$ at home in week $w$, 0 otherwise

$y_{wo}$ = 1 if we play team $o$ away in week $w$, 0 otherwise

$I$ = Set of all opponents

$W$ = Set of all weeks


\[
\begin{align}
\text{Maximize} \quad & \sum_{w \in W} \sum_{o \in I} HR_{wo} \cdot x_{wo} + AR_{wo} \cdot y_{wo} \\
\text{s.t.} \quad \notag
\end{align}
\]

\[
\begin{align*}
&\sum_{o \in I} (x_{wo} + y_{wo}) \leq 1 && \forall w \in W \quad \text{(Only one game per week)} \\
&\sum_{w \in W} (x_{wo} + y_{wo}) = 1 && \forall o \in I \quad \text{(Each opponent appears once)} \\
&\sum_{w \in W} \sum_{o \in I} x_{wo} = 6 && \text{(Total home games)} \\
&\sum_{w \in W} \sum_{o \in I} y_{wo} = 6 && \text{(Total away games)} \\
&x_{wo}, y_{wo} \in \{0, 1\} && \forall w \in W, o \in I
\end{align*}
\]

In [ ]:
%%writefile soccerbasic.mod
set Weeks := {1..12};
set Opponents := {"Ironclad FC", "Stormbreakers United", "Crimson Strikers", "Coastal Rovers", "Blazing Barracudas", "Highland Havoc", "Emerald Thunder"};

var x{w in Weeks, o in Opponents} binary;
var y{w in Weeks, o in Opponents} binary;

param HR{w in Weeks, o in Opponents};
param AR{w in Weeks, o in Opponents};

maximize Z: sum{w in Weeks, o in Opponents} (HR[w,o] * x[w,o]) + sum {w in Weeks, o in Opponents} (AR[w,o] * y[w,o]);

# Constraints
s.t. NoThreeConsecAway{w in 1..10}: sum{o in Opponents} y[w,o] + sum{o in Opponents} y[w+1,o] + sum{o in Opponents} y[w+2,o] <= 2;
s.t. NoFourConsecHome{w in 1..9}: sum{o in Opponents} x[w,o] + sum{o in Opponents} x[w+1,o] + sum{o in Opponents} x[w+2,o] + sum{o in Opponents} x[w+3,o] <= 3;
s.t. Week{w in Weeks}: sum{o in Opponents} (x[w,o] + y[w,o]) = 1;
s.t. HomeGames: sum{w in Weeks, o in Opponents} x[w,o] = 6;
s.t. Opponent{o in Opponents}: sum{w in Weeks} (x[w,o] + y[w,o]) <= 2;




Writing soccerbasic.mod


In [ ]:
%%writefile soccerbasic.dat
param HR: "Ironclad FC" "Stormbreakers United" "Crimson Strikers" "Coastal Rovers" "Blazing Barracudas" "Highland Havoc" "Emerald Thunder":=
1 7800 4500 9200 3300 8100 5900 9800
2 3100 7600 4200 8800 5300 9700 6900
3 9900 5500 8700 3100 6400 9100 4800
4 4400 8800 3300 7500 5600 9500 6700
5 8000 3400 6600 9300 4700 8900 3000
6 5800 9200 4100 8600 6300 9000 3500
7 9100 4700 8900 6400 9500 3200 7700
8 4200 8700 6800 9900 3000 7400 5100
9 7500 6200 9400 4900 8500 3600 7000
10 6100 9600 3700 7100 5200 8300 9700
11 3900 7000 5300 8200 9900 4400 6500
12 7200 3800 8400 5100 9000 6700 4300;

param AR: "Ironclad FC" "Stormbreakers United" "Crimson Strikers" "Coastal Rovers" "Blazing Barracudas" "Highland Havoc" "Emerald Thunder":=
1 3900 2250 4600 1650 4050 2950 4900
2 1550 3800 2100 4400 2650 4850 3450
3 4950 2750 4350 1550 3200 4550 2400
4 2200 4400 1650 3750 2800 4800 3350
5 4000 1700 3300 4650 2350 4450 1500
6 2900 4600 2050 4300 3150 4500 1750
7 4550 2350 4450 3200 4750 1600 3850
8 2100 4350 3400 4950 1500 3700 2550
9 3750 3100 4700 2450 4250 1800 3500
10 3050 4800 1850 3550 2600 4150 4850
11 1950 3500 2650 4100 4950 2200 3250
12 3600 1900 4200 2550 4500 3350 2150;


Writing soccerbasic.dat


In [ ]:
%%ampl_eval

reset;

model soccerbasic.mod;
data soccerbasic.dat;

option solver cbc;

solve;

display Z,x,y, HR, AR ;

cbc 2.10.12: cbc 2.10.12: optimal solution; objective 86500
0 simplex iterations
Z = 86500

:                           x   y    HR     AR     :=
1  'Blazing Barracudas'     0   0   8100   4050
1  'Coastal Rovers'         0   0   3300   1650
1  'Crimson Strikers'       0   0   9200   4600
1  'Emerald Thunder'        1   0   9800   4900
1  'Highland Havoc'         0   0   5900   2950
1  'Ironclad FC'            0   0   7800   3900
1  'Stormbreakers United'   0   0   4500   2250
2  'Blazing Barracudas'     0   0   5300   2650
2  'Coastal Rovers'         0   0   8800   4400
2  'Crimson Strikers'       0   0   4200   2100
2  'Emerald Thunder'        0   0   6900   3450
2  'Highland Havoc'         0   1   9700   4850
2  'Ironclad FC'            0   0   3100   1550
2  'Stormbreakers United'   0   0   7600   3800
3  'Blazing Barracudas'     0   0   6400   3200
3  'Coastal Rovers'         0   0   3100   1550
3  'Crimson Strikers'       0   0   8700   4350
3  'Emerald Thunder'     